<a href="https://colab.research.google.com/github/TavionK/CPSC-419/blob/main/projects/Bach/Bach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#get the data
import pandas as pd
bach = pd.read_csv('https://raw.githubusercontent.com/TavionK/CPSC-419/main/projects/Bach/bach.csv')
#fix the notes and bass columns
bach['c'] = bach['C'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('C', axis=1)
bach['c#'] = bach['C#'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('C#', axis=1)
bach['d'] = bach['D'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('D', axis=1)
bach['d#'] = bach['D#'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('D#', axis=1)
bach['e'] = bach['E'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('E', axis=1)
bach['f'] = bach['F'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('F', axis=1)
bach['f#'] = bach['F#'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('F#', axis=1)
bach['g'] = bach['G'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('G', axis=1)
bach['g#'] = bach['G#'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('G#', axis=1)
bach['a'] = bach['A'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('A', axis=1)
bach['a#'] = bach['A#'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('A#', axis=1)
bach['b'] = bach['B'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('B', axis=1)
one_hot = pd.get_dummies(bach['bass'])
bach = bach.drop('bass', axis=1)
bach = bach.join(one_hot)
print(bach)


     choral_ID  event_number  meter chord_label      c  ...  Eb  F  F#  G  G#
0     000106b_             1      3         F_M   True  ...   0  1   0  0   0
1     000106b_             2      5         C_M   True  ...   0  0   0  0   0
2     000106b_             3      2         C_M   True  ...   0  0   0  0   0
3     000106b_             4      3         F_M   True  ...   0  1   0  0   0
4     000106b_             5      2         F_M   True  ...   0  1   0  0   0
...        ...           ...    ...         ...    ...  ...  .. ..  .. ..  ..
5660  015505b_           105      4         G_m  False  ...   0  0   0  1   0
5661  015505b_           106      3         G_m  False  ...   0  0   0  1   0
5662  015505b_           107      5         C_M   True  ...   0  0   0  0   0
5663  015505b_           108      3         C_M   True  ...   0  0   0  0   0
5664  015505b_           109      4         F_M  False  ...   0  1   0  0   0

[5665 rows x 32 columns]


In [26]:
bach.set_index('choral_ID', inplace=True)
bFeatures = bach.drop('chord_label', axis=1)
bLabels = bach['chord_label']
from sklearn.model_selection import train_test_split
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bFeatures, bLabels, test_size = 0.2, random_state=42)
print(bach)
#create the clf's
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(bach_train_features,bach_train_labels)

           event_number  meter chord_label      c     c#  ...  Eb  F  F#  G  G#
choral_ID                                                 ...                  
000106b_              1      3         F_M   True  False  ...   0  1   0  0   0
000106b_              2      5         C_M   True  False  ...   0  0   0  0   0
000106b_              3      2         C_M   True  False  ...   0  0   0  0   0
000106b_              4      3         F_M   True  False  ...   0  1   0  0   0
000106b_              5      2         F_M   True  False  ...   0  1   0  0   0
...                 ...    ...         ...    ...    ...  ...  .. ..  .. ..  ..
015505b_            105      4         G_m  False  False  ...   0  0   0  1   0
015505b_            106      3         G_m  False  False  ...   0  0   0  1   0
015505b_            107      5         C_M   True  False  ...   0  0   0  0   0
015505b_            108      3         C_M   True  False  ...   0  0   0  0   0
015505b_            109      4         F

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [34]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, bach_train_features, bach_train_labels, cv=10)
print(scores)
print("Mean is %5.3f" % (scores.mean()))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


[0.67621145 0.68502203 0.68432671 0.66225166 0.68432671 0.72406181
 0.70860927 0.64900662 0.67549669 0.70640177]
Mean is 0.686


In [57]:
#find the best depth
from sklearn.model_selection import GridSearchCV
hyperparam_grid = [
    {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
     'min_samples_split': [2,3,4, 5]}
  ]
clf = tree.DecisionTreeClassifier(criterion='entropy')
grid_search = GridSearchCV(clf, hyperparam_grid, cv=10)
grid_search.fit(bach_train_features, bach_train_labels)
grid_search.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


{'max_depth': 10, 'min_samples_split': 5}

In [59]:
from sklearn.metrics import accuracy_score
predictions = grid_search.best_estimator_.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.7007943512797882